# Python AST
- Reference: https://docs.python.org/3.10/library/ast.html#abstract-grammar

In [2]:
import ast

## Literals

In [10]:
print(ast.dump(ast.parse('123', mode='eval')))

Expression(body=Constant(value=123))


In [12]:
print(ast.dump(ast.parse('f"sin({a}) is {sin(a):.3}"', mode='eval'), indent=4))

Expression(
    body=JoinedStr(
        values=[
            Constant(value='sin('),
            FormattedValue(
                value=Name(id='a', ctx=Load()),
                conversion=-1),
            Constant(value=') is '),
            FormattedValue(
                value=Call(
                    func=Name(id='sin', ctx=Load()),
                    args=[
                        Name(id='a', ctx=Load())],
                    keywords=[]),
                conversion=-1,
                format_spec=JoinedStr(
                    values=[
                        Constant(value='.3')]))]))


In [20]:
print(ast.dump(ast.parse('[1, 2, 3]', mode='eval'), indent=4))

Expression(
    body=List(
        elts=[
            Constant(value=1),
            Constant(value=2),
            Constant(value=3)],
        ctx=Load()))


In [21]:
print(ast.dump(ast.parse('(1, 2, 3)', mode='eval'), indent=4))

Expression(
    body=Tuple(
        elts=[
            Constant(value=1),
            Constant(value=2),
            Constant(value=3)],
        ctx=Load()))


In [22]:
print(ast.dump(ast.parse('{1, 2, 3}', mode='eval'), indent=4))

Expression(
    body=Set(
        elts=[
            Constant(value=1),
            Constant(value=2),
            Constant(value=3)]))


In [23]:
print(ast.dump(ast.parse('{"a":1, **d}', mode='eval'), indent=4))

Expression(
    body=Dict(
        keys=[
            Constant(value='a'),
            None],
        values=[
            Constant(value=1),
            Name(id='d', ctx=Load())]))


## Variables

- `ast.Expr`, `ast.Assign`, `ast.Delete`, `ast.Starred`.
- Also has `ast.Store`, `ast.Load` for storing and loading values in **Literals**.

In [ ]:
print(ast.dump(ast.parse('a'), indent=4))

Module(
    body=[
        Expr(
            value=Name(id='a', ctx=Load()))],
    type_ignores=[])


In [25]:
print(ast.dump(ast.parse('a = 1'), indent=4))

Module(
    body=[
        Assign(
            targets=[
                Name(id='a', ctx=Store())],
            value=Constant(value=1))],
    type_ignores=[])


In [26]:
print(ast.dump(ast.parse('del a'), indent=4))

Module(
    body=[
        Delete(
            targets=[
                Name(id='a', ctx=Del())])],
    type_ignores=[])


In [27]:
print(ast.dump(ast.parse('a, *b = it'), indent=4))

Module(
    body=[
        Assign(
            targets=[
                Tuple(
                    elts=[
                        Name(id='a', ctx=Store()),
                        Starred(
                            value=Name(id='b', ctx=Store()),
                            ctx=Store())],
                    ctx=Store())],
            value=Name(id='it', ctx=Load()))],
    type_ignores=[])


## Expressions

In [6]:
print(ast.dump(ast.parse('func(a, b=c, *d, **e)', mode='eval'), indent=4))

Expression(
    body=Call(
        func=Name(id='func', ctx=Load()),
        args=[
            Name(id='a', ctx=Load()),
            Starred(
                value=Name(id='d', ctx=Load()),
                ctx=Load())],
        keywords=[
            keyword(
                arg='b',
                value=Name(id='c', ctx=Load())),
            keyword(
                value=Name(id='e', ctx=Load()))]))


In [7]:
test = ast.parse("a=1; func1(a)", mode="exec")

In [8]:
dir(test.body)

['__add__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [9]:
test.body

[<ast.Assign at 0x7e234c31cd60>, <ast.Expr at 0x7e234c31ceb0>]

In [10]:
test.body[0].targets[0].id

'a'

In [14]:
test.body[0].lineno

1

In [23]:
test.body[1].value.func.id

'func1'

In [33]:
test.body[1].value.args[0].id

'a'

## Strange Snippet Testing

In [15]:
print(ast.dump(ast.parse('[a.append(line) for line in file]', mode='eval'), indent=4))

Expression(
    body=ListComp(
        elt=Call(
            func=Attribute(
                value=Name(id='a', ctx=Load()),
                attr='append',
                ctx=Load()),
            args=[
                Name(id='line', ctx=Load())],
            keywords=[]),
        generators=[
            comprehension(
                target=Name(id='line', ctx=Store()),
                iter=Name(id='file', ctx=Load()),
                ifs=[],
                is_async=0)]))


# Questions

### How to Distinguish Passed by Value and Passed by Reference?

Do I need this?

In [29]:
code = """
t = []

def func(t):
    t.append(1)
    
func(t)"""

In [30]:
print(ast.dump(ast.parse(code, mode="exec"), indent=4))

Module(
    body=[
        Assign(
            targets=[
                Name(id='t', ctx=Store())],
            value=List(elts=[], ctx=Load())),
        FunctionDef(
            name='func',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(arg='t')],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                Expr(
                    value=Call(
                        func=Attribute(
                            value=Name(id='t', ctx=Load()),
                            attr='append',
                            ctx=Load()),
                        args=[
                            Constant(value=1)],
                        keywords=[]))],
            decorator_list=[]),
        Expr(
            value=Call(
                func=Name(id='func', ctx=Load()),
                args=[
                    Name(id='t', ctx=Load())],
                keywords=[]))]

In [31]:
del(t)

NameError: name 't' is not defined